In [2]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.models import model_from_json
import numpy as np
import subprocess
import os

# Moving images to validation folder

In [153]:
# b_path = '/home/nikita/Roman_Num/New_data/Train/'
# list_classes = os.listdir('/home/nikita/Roman_Num/New_data/Train/')
# # count_el = 0
# for class_name in list_classes:
#     count_el = 0
# #     print(class_name)
#     for element in os.listdir(b_path + class_name + '/'):
#         if not os.path.exists("/home/nikita/Roman_Num/New_data/Validation/" + class_name):
#             os.makedirs("/home/nikita/Roman_Num/New_data/Validation/" + class_name)
# #         print(element)
#         if count_el < 50:
#             print("mv", b_path + class_name + '/' + element, "/home/nikita/Roman_Num/New_data/Validation/" + class_name + '/')
#             subprocess.call(["mv", b_path + class_name + '/' + element, "/home/nikita/Roman_Num/New_data/Validation/" + class_name + '/'])
#             count_el +=1 

In [3]:
def get_count_elements(status):
    counter = 0
    if status == "Train":
        base_path = "/home/nikita/Roman_Num/New_data/Train/"
    if status == "Val":
        base_path = "/home/nikita/Roman_Num/New_data/Validation/"
    if status == "Test":
        base_path = "/home/nikita/Roman_Num/New_data/Test/"
    for class_ in os.listdir(base_path):
        count_el = len(os.listdir(base_path + class_))
        counter+=count_el
    print("Samples for {0}: {1}".format(base_path, counter))
    return counter

In [4]:
train_dir = '/home/nikita/Roman_Num/New_data/Train'

val_dir = '/home/nikita/Roman_Num/New_data/Validation'

test_dir = '/home/nikita/Roman_Num/New_data/Test'

img_width, img_height = 28, 28
input_shape = (img_width, img_height, 3)

epochs = 30

batch_size = 16

nb_train_samples = get_count_elements(status="Train")
nb_validation_samples = get_count_elements(status="Val")
nb_test_samples = get_count_elements(status="Test")


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(8))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

Samples for /home/nikita/Roman_Num/New_data/Train/: 6512
Samples for /home/nikita/Roman_Num/New_data/Validation/: 400
Samples for /home/nikita/Roman_Num/New_data/Test/: 1680
Found 6512 images belonging to 8 classes.


In [5]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

Found 400 images belonging to 8 classes.


In [6]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

Found 1680 images belonging to 8 classes.


In [7]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/30
407/407 [==============================] - 62s 153ms/step - loss: 1.5046 - acc: 0.4796 - val_loss: 0.3806 - val_acc: 0.8975
Epoch 2/30
407/407 [==============================] - 12s 30ms/step - loss: 0.2622 - acc: 0.9261 - val_loss: 0.0276 - val_acc: 0.9925
Epoch 3/30
407/407 [==============================] - 12s 29ms/step - loss: 0.1065 - acc: 0.9693 - val_loss: 0.0235 - val_acc: 0.9950
Epoch 4/30
407/407 [==============================] - 11s 26ms/step - loss: 0.0592 - acc: 0.9845 - val_loss: 0.0146 - val_acc: 0.9975
Epoch 5/30
407/407 [==============================] - 10s 26ms/step - loss: 0.0490 - acc: 0.9873 - val_loss: 0.0030 - val_acc: 1.0000
Epoch 6/30
407/407 [==============================] - 10s 26ms/step - loss: 0.0366 - acc: 0.9896 - val_loss: 0.0032 - val_acc: 1.0000
Epoch 7/30
407/407 [==============================] - 11s 27ms/step - loss: 0.0349 - acc: 0.9899 - val_loss: 0.0017 - val_acc: 1.0000
Epoch 8/30
407/407 [==============================] - 10s 26m

In [8]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

Аккуратность на тестовых данных: 99.94%


In [9]:
model_json = model.to_json()
# Записываем модель в файл
json_file = open("mnist_model.json", "w")
json_file.write(model_json)
json_file.close()

model_yaml = model.to_yaml()
yaml_file = open("mnist_model.yml", "w")
# Записываем модель в файл
yaml_file.write(model_yaml)
yaml_file.close()
model.save_weights("mnist_model.h5")

In [10]:
json_file = open("mnist_model.json", "r")
loaded_model_json = json_file.read()
json_file.close()
# Создаем модель на основе загруженных данных
loaded_model = model_from_json(loaded_model_json)
# Загружаем веса в модель
loaded_model.load_weights("mnist_model.h5")
loaded_model.compile(loss="sparse_categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])
# Проверяем модель на тестовых данных
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Точность модели на тестовых данных: %.2f%%" % (scores[1]*100))

Точность модели на тестовых данных: 99.94%


In [14]:
test_dir = 'ts'
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

Found 1 images belonging to 1 classes.


In [15]:
test_generator.reset()
predict = model.predict_generator(test_generator, steps = None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)
for i in predict:
    print(np.argmax(i))



6
